In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/My_Projects/app

/content/drive/MyDrive/My_Projects/app


In [1]:

pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#Data Preprocessing

In [11]:
import cv2,os
data_path = r'c:\Users\DeLL\Desktop\Crop Guru\Plant_leaf_diseases_dataset_with_augmentation\Plant_leaf_diseases_dataset_with_augmentation\Plant_leave_diseases_dataset_with_augmentation'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary
print(label_dict)
print(categories)
print(labels)

{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry___healthy': 5, 'Cherry___Powdery_mildew': 6, 'Chili__leaf curl': 7, 'Chili__yellowish': 8, 'Coffee__cercospora_leaf_spot': 9, 'Coffee__red_spider_mite': 10, 'Corn___Cercospora_leaf_spot Gray_leaf_spot': 11, 'Corn___Common_rust': 12, 'Corn___healthy': 13, 'Corn___Northern_Leaf_Blight': 14, 'Cucumber__diseased': 15, 'Gauva__diseased': 16, 'Grape__leaf_blight_(isariopsis_leaf_spot)': 17, 'Grape___Black_rot': 18, 'Grape___Esca_(Black_Measles)': 19, 'Grape___healthy': 20, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 21, 'Lemon__diseased': 22, 'Mango__diseased': 23, 'Orange___Haunglongbing_(Citrus_greening)': 24, 'Pepper,_bell___Bacterial_spot': 25, 'Pepper,_bell___healthy': 26, 'Potato___Early_blight': 27, 'Potato___healthy': 28, 'Potato___Late_blight': 29, 'Rice__brown_spot': 30, 'Rice__hispa': 31, 'Rice__neck_blast': 32, 'Soybean___healthy': 33, 'Squash

In [20]:
img_size=100
data=[]
target=[]

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:  
            resized=cv2.resize(img,(img_size,img_size))
            #resizing the image  into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)
        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [4]:
pip install numpy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#Recale and assign  catagorical lables

In [13]:
import numpy as np
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,3))
target=np.array(target)
from tensorflow.python.keras.utils import np_utils
new_target=np_utils.to_categorical(target)

In [14]:
new_target.shape

(64208, 56)

#CNN Model

In [15]:
data.shape

(64208, 100, 100, 3)

In [16]:
data.shape[1:]

(100, 100, 3)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten,Dropout
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(300,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(80,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(56,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 300)       8400      
                                                                 
 activation (Activation)     (None, 98, 98, 300)       0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 300)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 100)       270100    
                                                                 
 activation_1 (Activation)   (None, 47, 47, 100)       0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 100)       0         
 g2D)                                                   

#Splittiong data into traning and testing

In [5]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

MemoryError: Unable to allocate 12.9 GiB for an array with shape (57787, 100, 100, 3) and data type float64

In [10]:
train_data.shape

(5035, 50, 50, 3)

In [11]:
train_target.shape

(5035, 52)

In [12]:
#checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=10,validation_split=0.2)

Epoch 1/10
126/126 [==============================] - 322s 3s/step - loss: 2.8951 - accuracy: 0.3138 - val_loss: 2.4479 - val_accuracy: 0.3694
Epoch 2/10
126/126 [==============================] - 313s 2s/step - loss: 2.0506 - accuracy: 0.4471 - val_loss: 1.8413 - val_accuracy: 0.5035
Epoch 3/10
126/126 [==============================] - 310s 2s/step - loss: 1.5597 - accuracy: 0.5566 - val_loss: 1.3946 - val_accuracy: 0.5958
Epoch 4/10
126/126 [==============================] - 331s 3s/step - loss: 1.3117 - accuracy: 0.6209 - val_loss: 1.3556 - val_accuracy: 0.6127
Epoch 5/10
126/126 [==============================] - 242s 2s/step - loss: 1.0994 - accuracy: 0.6815 - val_loss: 1.1116 - val_accuracy: 0.6624
Epoch 6/10
126/126 [==============================] - 188s 1s/step - loss: 0.9245 - accuracy: 0.7150 - val_loss: 0.9605 - val_accuracy: 0.7200
Epoch 7/10
126/126 [==============================] - 196s 2s/step - loss: 0.7779 - accuracy: 0.7614 - val_loss: 0.8044 - val_accuracy: 0.7676

In [13]:
model.save('model.h5')

/home/student/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [16]:
pip install matplotlib

     |████████████████████████████████| 9.2 MB 171 kB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 968 kB/s eta 0:00:01
     |████████████████████████████████| 300 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.6 MB 713 kB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [17]:
# plot the training loss and accuracy
N = 20
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.savefig("CNN_Model")

NameError: name 'plt' is not defined

In [7]:
import tensorflow as tf

In [8]:
tf.__version__

'2.13.0'